In [2]:
import pandas as pd
import numpy as np
import pathlib
from pathlib import Path

this_path = pathlib.Path().resolve()

In [47]:
stat_types = ["offense", "passing", "rushing"]
off_headers = ["Player","Tm","Passing_Cmp","Passing_Att","Passing_Yds","Passing_TD","Passing_Int","Passing_Sk","Passing_LostYds","Passing_Lng","Passing_Rate","Rushing_Att","Rushing_Yds","Rushing_TD","Rushing_Lng","Receiving_Tgt","Receiving_Rec","Receiving_Yds","Receiving_TD","Receiving_Lng","Fmb","FL"]
off_frames = []
pass_frames = []
rush_frames = []
for week in range (1,19):
    for stat in stat_types:
        wk_path = this_path / "data" / "Week_{}".format(week) / "{}_{}.csv".format(week, stat)
        if wk_path.is_file():
            if stat == "offense":
                df_wk = pd.read_csv(wk_path, delimiter=",", skiprows=2, names= off_headers)
            else:
                df_wk = pd.read_csv(wk_path, delimiter=",", header=[0])
            df_wk["Week"] = week
            df_wk[["Player", "PlayerID"]] = df_wk["Player"].str.split("\\",expand=True)

            if stat == "offense":
                off_frames.append(df_wk)
            elif stat == "passing":
                pass_frames.append(df_wk)
            else:
                rush_frames.append(df_wk)

df_off = pd.concat(off_frames, ignore_index = True)
df_pass = pd.concat(pass_frames, ignore_index = True)
df_rush = pd.concat(rush_frames, ignore_index = True)

df_off.to_csv(this_path / "data" / "offense.csv")
df_pass.to_csv(this_path / "data" / "passing.csv")
df_rush.to_csv(this_path / "data" / "rushing.csv")

In [15]:
# Let's merge the RBs with the salary to get a yds/$ thing going
rbs_path = this_path / "data" / "all_rushing_season.csv"
salary_path = this_path / "data" / "rb_salary.csv"

rbs = pd.read_csv(rbs_path, delimiter =",")
salary = pd.read_csv(salary_path, delimiter =",")

# filter all non RBs from the rushing data
rbs = rbs[rbs["Pos"].str.casefold() == "rb"]
# strip the names from the IDs
rbs[["Player", "PlayerID"]] = rbs["Player"].str.split("\\",expand=True)
# And also those pesky + and * characters
rbs["Player"] = rbs["Player"].str.replace('[^a-zA-Z0-9  \'.-]', '')
# And finally make an inner join on the Player column
combined  = pd.merge(rbs, salary, on="Player", how='inner')
combined.to_csv(this_path / "data" / "rb_combined.csv")

     Rk           Player   Tm  Age_x Pos   G  GS  Att   Yds  TD  ...    Y/G  \
0     1  Jonathan Taylor  IND     22  RB  17  17  332  1811  18  ...  106.5   
1     2     Najee Harris  PIT     23  RB  17  17  307  1200   7  ...   70.6   
2     3        Joe Mixon  CIN     25  RB  16  16  292  1205  13  ...   75.3   
3     4   Antonio Gibson  WAS     23  RB  16  14  258  1037   7  ...   64.8   
4     5      Dalvin Cook  MIN     26  RB  13  13  249  1159   6  ...   89.2   
..  ...              ...  ...    ...  ..  ..  ..  ...   ...  ..  ...    ...   
69  118    Malcolm Brown  MIA     28  rb   7   3   33   125   1  ...   17.9   
70  133    Elijhaa Penny  NYG     28  rb  17   3   24    99   1  ...    5.8   
71  145      Chris Evans  CIN     24  rb  14   1   17    77   0  ...    5.5   
72  199  Demetric Felton  CLE     23  rb  16   2    7    24   0  ...    1.5   
73  291   Raheem Mostert  SFO     29  rb   1   1    2    20   0  ...   20.0   

    Fmb  PlayerID        Team  Age_y Total Value Av

C:\Users\HUBERS\AppData\Local\Temp/ipykernel_23468/2150829142.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  rbs["Player"] = rbs["Player"].str.replace('[^a-zA-Z0-9  \'.-]', '')
